# Initialisation

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('seaborn')

import ta
import plotly
import plotly.express as px
import plotly.graph_objects as go
import cufflinks as cf

from datetime import datetime
from arch import arch_model

In [47]:
from scipy.stats import linregress

In [48]:
# Load the raw data
df_l = pd.read_csv('./FX Data/GBPUSD.csv', names=['date','open','high','low','close'])

In [49]:
df = df_l[0:500]

In [50]:
len(df_l)


5180

# Charts

In [51]:
#fig_cdls = go.Figure(data=[go.Candlestick(x=df['Date'][0:100],open=df['Open'],high=df['High'],low=df['Low'],close=df['Close'])])
#fig_cdls.show()

In [52]:
#fig = px.line(df[0:500],x='Date',y='Close')
#fig.show()

In [53]:
# ind_EMA = ta.trend.EMAIndicator(close=df['Close'],n=10,fillna=False)

# df['EMA'] = ind_EMA.ema_indicator()

In [54]:
# fig_ind = go.Figure()

# fig_ind.update_layout(xaxis_range=[0,50],
#                       yaxis_range=(1.5,1.7),
#                  title_text = "Close and EMA")

# fig_ind.add_trace(go.Scatter(
#                     x=df['Date'],
#                     y=df['Close'],
#                     name='Close',
#                     line_color='dimgray',
#                     opacity=0.4))

# fig_ind.add_trace(go.Scatter(
#                     x=df['Date'],
#                     y=df['EMA'],
#                     name='EMA',
#                     line_color='deepskyblue',
#                     opacity=0.4))

# fig_ind.show()

# Adding features
## Technical Analysis features

In [55]:
df['wclose'] = (df['close']*2+df['high']+df['low'])/4

In [56]:
for i in range(1,len(df)):
    if df.at[i,'high'] > df.at[i-1,'high']:
        df.at[i,'delta'] = 1
    else:
        df.at[i,'delta'] = 0

In [57]:
def add_SO(i):
    ind_SO = ta.momentum.StochasticOscillator(high=df['high'],low=df['low'],close=df['close'],n=i)
    name = 'so_%s' %(i)
    df[name] = ind_SO.stoch()
    return

for i in [3,4,5,8,9,10]:
    add_SO(i)

In [58]:
def add_WR(i):
    ind_WR = ta.momentum.WilliamsRIndicator(high=df['high'],low=df['low'],close=df['close'],lbp=i)
    name = 'wr_%s' %(i)
    df[name] = ind_WR.wr()
    return

for i in [6,7,8,9,10]:
    add_WR(i)

In [59]:
def add_ROC(i):
    ind_ROC = ta.momentum.ROCIndicator(close=df['close'],n=i)
    name = 'roc_%s' %(i)
    df[name] = ind_ROC.roc()
    return

for i in [12,13,14,15]:
    add_ROC(i)

In [60]:
def add_WCP(i):
    ind_WCP = ta.trend.EMAIndicator(close=df['wclose'],n=i)
    name = 'wcp_%s' %(i)
    df[name] = ind_WCP.ema_indicator()
    return

add_WCP(15)

In [61]:
def add_MACD(i,j):
    ind_MACD = ta.trend.MACD(close=df['close'],n_fast=i,n_slow=j)
    name ='macd_%s_%s' %(i,j)
    df[name] = ind_MACD.macd()
    return

add_MACD(15,30)  

In [62]:
def add_CCI(i):
    ind_CCI = ta.trend.cci(high=df['high'],low=df['low'],close=df['close'],n=i)
    name = 'cci_%s' %(i)
    df[name] = ind_CCI
    return

add_CCI(15)

## Signal Processing features

In [63]:
for i in range(1,len(df)):
    df.at[i,'hi_avg_2'] =  (df.at[i-1,'high'] + df.at[i,'high'])/2
    df.at[i,'lo_avg_2'] =  (df.at[i-1,'low'] + df.at[i,'low'])/2
    df.at[i,'hilo_avg_2'] = (df.at[i,'hi_avg_2'] + df.at[i,'lo_avg_2'])/2
    df.at[i,'hilo_avg'] = (df.at[i,'high'] + df.at[i,'low'])/2

In [64]:
def add_slope(i):
    for j in range(i-1,len(df)):
        a=df['high'][j-(i-1):j+1]
        b=[]
        for k in range(i):
            b.append(k)
        name = 'slope_%s' %(i)
        df.at[j,name] = linregress(a,b).slope
    
for i in [3,4,5,10,20,30]:
    add_slope(i)

In [65]:
df.head(n=10)

,date,open,high,low,close,wclose,delta,so_3,so_4,so_5,...,hi_avg_2,lo_avg_2,hilo_avg_2,hilo_avg,slope_3,slope_4,slope_5,slope_10,slope_20,slope_30
0,3/1/2000,1.6147,1.6396,1.6145,1.6358,1.631425,NaN,84.860558,84.860558,84.860558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4/1/2000,1.6358,1.6413,1.6316,1.6373,1.636875,1.0,85.074627,85.074627,85.074627,...,1.64045,1.62305,1.631750,1.63645,NaN,NaN,NaN,NaN,NaN,NaN
2,5/1/2000,1.6373,1.6451,1.6352,1.6419,1.641025,1.0,89.542484,89.542484,89.542484,...,1.64320,1.63340,1.638300,1.64015,346.784363,NaN,NaN,NaN,NaN,NaN
3,6/1/2000,1.6419,1.6509,1.6412,1.6477,1.646875,1.0,83.419689,91.208791,91.208791,...,1.64800,1.63820,1.643100,1.64605,205.362236,250.440097,NaN,NaN,NaN,NaN
4,7/1/2000,1.6477,1.6498,1.6361,1.6388,1.640875,0.0,22.929936,37.305699,66.758242,...,1.65035,1.63865,1.644500,1.64295,247.629083,265.490479,299.604522,NaN,NaN,NaN
5,10/1/2000,1.6388,1.6400,1.6335,1.6369,1.636825,0.0,19.540230,19.540230,27.461140,...,1.64490,1.63480,1.639850,1.63675,-151.346848,-110.141034,21.950914,NaN,NaN,NaN
6,11/1/2000,1.6369,1.6491,1.6350,1.6468,1.644425,1.0,81.595092,76.436782,76.436782,...,1.64455,1.63425,1.639400,1.64205,-11.709602,-100.462657,-36.219214,NaN,NaN,NaN
7,12/1/2000,1.6468,1.6508,1.6454,1.6471,1.647600,1.0,78.612717,78.612717,78.160920,...,1.64995,1.64020,1.645075,1.64810,160.126520,80.701637,-10.634778,NaN,NaN,NaN
8,13/1/2000,1.6471,1.6499,1.6412,1.6469,1.646225,0.0,75.316456,77.456647,77.456647,...,1.65035,1.64330,1.646825,1.64555,552.995392,208.084824,137.727250,NaN,NaN,NaN
9,14/1/2000,1.6469,1.6479,1.6313,1.6346,1.637100,0.0,16.923077,16.923077,16.923077,...,1.64890,1.63625,1.642575,1.63960,-658.093797,-494.777350,219.541872,394.476268,NaN,NaN


# Classifiers

<img src="files/algorithm cheat sheet.png">

In [24]:
from sklearn.svm import SVC

## RBF Classifier

In [43]:
training_labels =df['close']

In [45]:
type(training_labels)

pandas.core.series.Series

In [38]:
feature_matrix=df.drop(columns=['date','open','high','low','close','wclose','delta'])

In [41]:
feature_matrix['low']

0      1.6145
1      1.6316
2      1.6352
3      1.6412
4      1.6361
        ...  
495    1.4101
496    1.4150
497    1.4136
498    1.4211
499    1.4222
Name: low, Length: 500, dtype: float64